In [3]:
!git clone https://github.com/ultralytics/yolov5  # clone

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [5]:
%cd yolov5

/content/yolov5


In [8]:
%pip install -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.3.0 which is incompatible.


In [9]:
!python3 ./detect.py --weights ./best_face.pt --conf 0.5 --source ./test1.jpg --save-txt

detect: weights=['./best.pt'], source=./test1.jpg, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-297-gd07d0cf6 Python-3.10.12 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 182 layers, 7246518 parameters, 0 gradients
image 1/1 /content/yolov5/test1.jpg: 480x640 80 faces, 498.9ms
Speed: 4.0ms pre-process, 498.9ms inference, 24.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp
1 labels saved to runs/detect/exp/labels


In [11]:
import cv2

# 이미지 파일을 불러오는 함수
def load_image(image_path):
    return cv2.imread(image_path)

# txt 파일에서 바운딩 박스를 읽는 함수
def read_bboxes(txt_path, img_width, img_height):
    bboxes = []
    with open(txt_path, 'r') as file:
        for line in file:
            class_id, x_center, y_center, width, height = map(float, line.split())
            x1 = int((x_center - width / 2) * img_width)
            y1 = int((y_center - height / 2) * img_height)
            x2 = int((x_center + width / 2) * img_width)
            y2 = int((y_center + height / 2) * img_height)
            bboxes.append((x1, y1, x2, y2))
    return bboxes

# 모자이크 처리 함수
def apply_mosaic(img, bbox, mosaic_size):
    x1, y1, x2, y2 = bbox
    roi = img[y1:y2, x1:x2]  # 관심영역(Region of Interest) 추출
    roi = cv2.resize(roi, (mosaic_size, mosaic_size), interpolation=cv2.INTER_LINEAR)
    roi = cv2.resize(roi, (x2 - x1, y2 - y1), interpolation=cv2.INTER_NEAREST)
    img[y1:y2, x1:x2] = roi  # 원본 이미지에 모자이크 처리된 영역을 대체
    return img

# 이미지 경로와 txt 파일 경로
image_path = 'test1.jpg'
txt_path = './runs/detect/exp/labels/test1.txt'

# 이미지 로드
image = load_image(image_path)
img_height, img_width = image.shape[:2]

# 모자이크 크기 설정
mosaic_size = 10

# 바운딩 박스 정보 읽기
bboxes = read_bboxes(txt_path, img_width, img_height)

# 각 바운딩 박스에 대해 모자이크 처리 적용
for bbox in bboxes:
    image = apply_mosaic(image, bbox, mosaic_size)

# 모자이크 처리된 이미지 저장
cv2.imwrite('./mosiaic_test1.jpg', image)

True